# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [20]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset

from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

from azureml.core.webservice import LocalWebservice, AciWebservice
from azureml.core.model import Model
import requests
import json

from azureml.core import ScriptRunConfig

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


### Workspace details

In [21]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-capstone")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-134184
Azure region: southcentralus
Subscription id: 510b94ba-e453-4417-988b-fbdc37b55ca7
Resource group: aml-quickstarts-134184


### Create compute cluster

In [22]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

amlcompute_cluster_name = "project"

try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)


Creating
Succeeded.................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.


Next we used an indepandent train.py file to first get the data extracted from https://github.com/Aishwaryasasanapuri/test/blob/main/Capstone_file/heart_failure_clinical_records_dataset.csv , then define the parameters (C , max_iter), then clean the data (by removing the name of the personne and defining the dependent and independent variable) and finally set the training and the testing data

In [24]:
from azureml.core import Dataset

df = Dataset.get_by_name(ws, name='heart-disease Dataset')
df= df.to_pandas_dataframe()
df

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,62.0,0,61,1,38,1,155000.00,1.1,143,1,1,270,0
295,55.0,0,1820,0,38,0,270000.00,1.2,139,0,0,271,0
296,45.0,0,2060,1,60,0,742000.00,0.8,138,0,0,278,0
297,45.0,0,2413,0,38,0,140000.00,1.4,140,1,1,280,0


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

Logistic Regression is a binary classification algorithm(0 or 1).  It uses logistic function called the sigmoid function in order to predict outcomes. 

And To improve the model we optimize the hyperparameters using Azure Machine Learning's tuning capabilities Hyperdrive

 **RandomParameterSampling** parameter sampler is used where parameter values are randomly chosen from a set of discrete values or a distribution over a continuous range. 
 
 **--C**  have a uniform distribution with 0.5 as a minimum value and 1.5 as a maximum value
 **--max-iter** will be a choice 


During hyperdrive configuration, **BanditPolicy** is chosen as an early termination policy. 
An finally we define the hyperdrive configuration where we set 50 as the maximum of iteration before submiting the experiment

In [25]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import choice
from azureml.core import ScriptRunConfig
import os

# Specify parameter sampler
ps =  RandomParameterSampling( {
        "--C": uniform(0.5, 1.5),
        "--max_iter": choice(16, 32, 64, 128)
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=2)

if "training" not in os.listdir():
    os.mkdir("./training")
sklearn_env = Environment.get(workspace=ws, name="AzureML-Tutorial")
# Create a SKLearn estimator for use with train.py
src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target = compute_target,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=50,
                             max_concurrent_runs=4)

In [26]:

hyperdrive_run = exp.submit(hyperdrive_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?
Different types of Models were trained for every combination of hyperparameters (RandomParameterSampling). 

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [27]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [28]:
import joblib

# best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print(best_run.get_details()['runDefinition']['arguments'])
print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])


os.makedirs('outputs', exist_ok=True)

#registering the model
model = best_run.register_model(model_name='logistic_hd_run',model_path='outputs/hypermodel.pkl',model_framework=Model.Framework.SCIKITLEARN,)
print(best_run.get_file_names())
print(best_run.get_tags())

['--C', '0.6912176661286786', '--max_iter', '128']
Best Run Id:  HD_11a0913d-2810-40c0-a1ac-016404f795b8_30

 Accuracy: 0.8333333333333334
['azureml-logs/55_azureml-execution-tvmps_ac1e5757d96bab245ba511e7aa967c9041d41dfe193be7f9634d883a1ed8bae9_d.txt', 'azureml-logs/65_job_prep-tvmps_ac1e5757d96bab245ba511e7aa967c9041d41dfe193be7f9634d883a1ed8bae9_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_ac1e5757d96bab245ba511e7aa967c9041d41dfe193be7f9634d883a1ed8bae9_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/104_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/engine_spans_8def14c9-3cab-4cc2-bbd8-3d0c621f8ad3.jsonl', 'logs/azureml/dataprep/python_span_8def14c9-3cab-4cc2-bbd8-3d0c621f8ad3.jsonl', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/hypermodel.pkl']
{'_aml_system_ComputeTargetStatu

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [37]:
%%writefile score.py
import json
import numpy as np
import os
import pickle
import joblib
import pandas as pd

def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'hypermodel.pkl')
    model = joblib.load(model_path)

def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        data = pd.DataFrame.from_dict(data)
        # make prediction
        result = model.predict(data)
        # Log the input and output data to appinsights:
        info = {
            "input": raw_data,
            "output": result.tolist()
            }
        print(json.dumps(info))
        return result.tolist()
    except Exception as ex:
        error = str(ex)
        return error

Overwriting score.py


In [38]:
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.model import Model

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               enable_app_insights=True, auth_enabled=True,
                                               tags={"data": "heart",  "method" : "sklearn"},  description='Heart-Failure-Prediction')



ws = Workspace.from_config()
model = Model(ws, 'logistic_hd_run')


inference_config = InferenceConfig(entry_script="score.py", environment=sklearn_env)

service_name = 'hyper-logistic'
service = Model.deploy(workspace=ws, 
                       name=service_name, 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aciconfig)

service.wait_for_deployment(show_output=True)



print(service.state)
print("scoring URI: " + service.scoring_uri)
print(service.get_logs())



Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..............
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
scoring URI: http://5c1dd3d2-1561-40ed-9261-8ca697996a75.southcentralus.azurecontainer.io/score
2021-01-10T10:20:30,321297800+00:00 - rsyslog/run 
2021-01-10T10:20:30,327210300+00:00 - gunicorn/run 
2021-01-10T10:20:30,322962100+00:00 - iot-server/run 
2021-01-10T10:20:30,364648500+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_a1736710baabf05d37337a101fa8d6d1/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a1736710baabf05d37337a101fa8d6d1/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a1736710baabf05d37337a101fa8d6d1/lib/libssl.so.1.0.0: no v

TODO: In the cell below, send a request to the web service you deployed to test it.

In [39]:
primary, secondary = service.get_keys()
print(primary,'\n',secondary)

UMCw4PEMsXnJAodzF2D9ycRopyXDSFsF 
 Pg4ZBOGghqvNVhzI9a78yXu5VnMdHN4y


In [40]:
print("scoring URI: " + service.scoring_uri)

print("Swagger URI: " + service.swagger_uri)

print("Authetication Key: " + service.get_keys()[0])

scoring URI: http://5c1dd3d2-1561-40ed-9261-8ca697996a75.southcentralus.azurecontainer.io/score
Swagger URI: http://5c1dd3d2-1561-40ed-9261-8ca697996a75.southcentralus.azurecontainer.io/swagger.json
Authetication Key: UMCw4PEMsXnJAodzF2D9ycRopyXDSFsF


In [41]:
import requests
import json
scoringuri = service.scoring_uri
key=primary
data= { "data":
       [
           {
               'age': 60,
               'anaemia': 245,
               'creatinine_phosphokinase': 0,
               'diabetes': 0,
               'ejection_fraction': 38,
               'high_blood_pressure': 1,
               'platelets': 163000,
               'serum_creatinine': 50,
               'serum_sodium':100,
               'sex':1,
               'smoking':1,
               'time':7
               
               
           }
       ]
    }
input_data = json.dumps(data)

headers = {'Content-Type': 'application/json'}
headers['Authorization'] = f'Bearer {key}'

response = requests.post(scoringuri, input_data, headers = headers)
print(response.text)

[1]


In [42]:
import json


payload = json.dumps({
    'data':
           [
               {
                   'age': 60,
               'anaemia': 245,
               'creatinine_phosphokinase': 0,
               'diabetes': 0,
               'ejection_fraction': 38,
               'high_blood_pressure': 1,
               'platelets': 163000,
               'serum_creatinine': 50,
               'serum_sodium':100,
               'sex':1,
               'smoking':1,
               'time':7

               }
           ],
    'method': 'predict_proba'  
    # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(payload)

print(output)

[1]


TODO: In the cell below, print the logs of the web service and delete the service

In [43]:
# print logs
from azureml.core import Workspace
from azureml.core.webservice import Webservice

# Requires the config to be downloaded first to the current working directory
ws = Workspace.from_config()

# Set with the deployment name
name = "hyper-logistic"

# load existing web service
aks_service = Webservice(name=name, workspace=ws)
aks_service.update(enable_app_insights=True)
logs = service.get_logs()

for line in logs.split('\n'):
    print(line)

2021-01-10T10:20:30,321297800+00:00 - rsyslog/run 
2021-01-10T10:20:30,327210300+00:00 - gunicorn/run 
2021-01-10T10:20:30,322962100+00:00 - iot-server/run 
2021-01-10T10:20:30,364648500+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_a1736710baabf05d37337a101fa8d6d1/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a1736710baabf05d37337a101fa8d6d1/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a1736710baabf05d37337a101fa8d6d1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a1736710baabf05d37337a101fa8d6d1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a1736710baabf05d37337a101fa8d6d1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [44]:
service.delete()
aks_service.delete()
print(service,'\n',aks_service)

No service with name hyper-logistic found to delete.
AciWebservice(workspace=Workspace.create(name='quick-starts-ws-134184', subscription_id='510b94ba-e453-4417-988b-fbdc37b55ca7', resource_group='aml-quickstarts-134184'), name=hyper-logistic, image_id=None, compute_type=None, state=ACI, scoring_uri=Deleting, tags=http://5c1dd3d2-1561-40ed-9261-8ca697996a75.southcentralus.azurecontainer.io/score, properties={'data': 'heart', 'method': 'sklearn'}, created_by={'hasInferenceSchema': 'False', 'hasHttps': 'False'}) 
 AciWebservice(workspace=Workspace.create(name='quick-starts-ws-134184', subscription_id='510b94ba-e453-4417-988b-fbdc37b55ca7', resource_group='aml-quickstarts-134184'), name=hyper-logistic, image_id=None, compute_type=None, state=ACI, scoring_uri=Healthy, tags=http://5c1dd3d2-1561-40ed-9261-8ca697996a75.southcentralus.azurecontainer.io/score, properties={'data': 'heart', 'method': 'sklearn'}, created_by={'hasInferenceSchema': 'False', 'hasHttps': 'False'})


In [45]:
compute_target.delete()

Current provisioning state of AmlCompute is "Deleting"

